# Twitter Pre-processing

Twitter Pre-processing to build following:

1. Twitter Timeline Data
2. Twitter User Profile Data
3. Twitter Most Frequent Words Data
4. Twitter Most Frequent Hashtags Data
5. Twitter Most Frequent Mentions Data

In [131]:
import json
import requests
from requests_oauthlib import OAuth1
import json
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import tweepy
import seaborn as sns
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn')

In [132]:
campus = "iupui"
twitter_file = "indiana-university_purdue-university-indianapolis"

## Check no of files Raw data

In [133]:
raw_data_dir = f"../../data/raw_data/{campus}/twitter/"
raw_files = [i for i in os.listdir(raw_data_dir)]
print(raw_files)

os.listdir(raw_data_dir)

['indiana-university_purdue-university-indianapolis_twitter_data.csv', 'iupui_twitter_data.csv', '.DS_Store', 'indiana-university_purdue-university-indianapolis_twitter_user_profile_data.csv', 'iupui_twitter_user_profile_data.csv']


['indiana-university_purdue-university-indianapolis_twitter_data.csv',
 'iupui_twitter_data.csv',
 '.DS_Store',
 'indiana-university_purdue-university-indianapolis_twitter_user_profile_data.csv',
 'iupui_twitter_user_profile_data.csv']

## Settings

In [134]:
raw_data_dir = f"../../data/raw_data/{campus}/twitter/"
raw_file_data = f"{twitter_file}_twitter_data.csv"
raw_file_userinfo = f"{twitter_file}_twitter_user_profile_data.csv"


processed_data_dir =  f"../../data/processed_data/{campus}/twitter/"
file = f"{twitter_file}_twitter_data.xlsx"


####

### twitter data - main sheet
twitter_data_sheetname = "twitter_data"

## user profile sheet 
twitter_user_profile_data_sheetname = "user_profile_data"

## Most frequent words
most_frequent_words_sheetname = "most_freq_100_words"

## Most frequent hashtags
most_frequent_hashtags_sheetname = "most_freq_100_hashtags"

## Most frequent screennames
most_frequent_screennames_sheetname = "most_freq_100_mentions"

###



In [135]:
import nltk
stop_words =   nltk.corpus.stopwords.words('english') + [
 '.',
 ',',
 '--',
 '\'s',
 '?',
 ')',
 '(',
 ':',
 '\'',
 '\'re',
 '"',
 '-',
 '}',
 '{',
 u'—',
     
 ]

In [136]:
## Create Directory if it doesnt exists
def make_dir_if_not_exists(directory):
    dir_path = Path(directory)
    if not dir_path.exists():
        os.makedirs(dir_path)
    else:
        print("Path exists")
    
make_dir_if_not_exists(processed_data_dir)
    

Path exists


## Read Data

In [137]:
import shutil  

## raw file
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_data), os.path.join(processed_data_dir,raw_file_data))
#print("Copied data file!")
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_userinfo), os.path.join(processed_data_dir,raw_file_userinfo))
#print("Copied user file!")



In [138]:
## read data
import csv
df = pd.read_csv(os.path.join(raw_data_dir,raw_file_data))
user_data_df = pd.read_csv(os.path.join(raw_data_dir,raw_file_userinfo))

## save currentfile 
#df.to_excel(os.path.join(processed_data_dir,"indiana-university-bl_twitter_data.xls"),index=False)

df["created"] = pd.to_datetime(df["created"])

In [139]:
print("No of rows and columns: {}, min date: {}, max date: {}".format(df.shape,df['created'].min(),df['created'].max()))
      

No of rows and columns: (2649, 22), min date: 2018-07-31 13:01:59, max date: 2019-09-17 18:46:00


In [140]:
df.head()

,created,text,sentiment,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,user_statuses_count,...,favorite_count,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type,words
0,2019-09-17 18:46:00,YOU SUNK MY….\n\n🚢 @iupuidm Battleships\n🚢 8 p.m.\n🚢 Natatorium https://t.co/n7Q9Gj6lSG,Neutral,1174031766677598209,False,IUPUI,36452,3143,5413,27327,...,50,0,IUPUIdm,NaN,NaN,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet,sunk … battleship .. natatorium
1,2019-09-17 16:35:24,Tune in tomorrow!👇👇👇 https://t.co/d3UWRrhxTr,Neutral,1173998901914816512,False,IUPUI,36452,3143,5413,27327,...,5,0,NaN,NaN,https://t.co/d3UWRrhxTr,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,quote,tune tomorrow
2,2019-09-17 16:06:46,Happy 125th Birthday to our most \nlaw-some school: @IUMcKinney! 🥳 https://t.co/xnaQyus1WI,Neutral,1173991695945367552,False,IUPUI,36452,3143,5413,27327,...,38,0,IUMcKinney,NaN,https://t.co/xnaQyus1WI,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,quote,happi 125th birthday law-som school
3,2019-09-17 15:11:33,Mom: Are you getting involved in college?\n\nYou: https://t.co/Ek2hweY6Pm,Neutral,1173977797896880129,False,IUPUI,36452,3143,5413,27327,...,246,0,NaN,NaN,https://t.co/Ek2hweY6Pm,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,quote,mom get involv colleg
4,2019-09-17 12:05:17,RT @IUPUIRegatta: FOUR days till Regatta! Here’s what we have going on today. #YouGottaRegatta https://t.co/HF5a3MVypY,Neutral,1173930924628697088,False,IUPUI,36452,3143,5413,27327,...,0,0,IUPUIRegatta,YouGottaRegatta,NaN,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,retweet,four day till regatta ’ go today


In [141]:
pd.set_option('display.max_colwidth',-1)



## Most Frequent Words


In [142]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer

def tweet_tokenizer(txt):
    try:
        tokenizer = TweetTokenizer()
        all_tokens = tokenizer.tokenize(txt.lower())
        # this line filters out all tokens that are entirely non-alphabetic characters
        filtered_tokens = [t for t in all_tokens if t.islower()]
        # filter out all tokens that are <=2 chars
        filtered_tokens = [x for x in filtered_tokens if len(x)>2]
    except IndexError:
        filtered_tokens = []
    return(filtered_tokens)


tweets_text = str(df["text"])

In [143]:
tweet_tokenizer(tweets_text)

['you',
 'sunk',
 '@iupuidm',
 'battleships',
 'natatorium',
 'https://t.co/n7q9gj6lsg',
 'tune',
 'tomorrow',
 'https://t.co/d3uwrrhxtr',
 'happy',
 '125th',
 'birthday',
 'our',
 'most',
 'nlaw-some',
 'school',
 '@iumckinney',
 'https://t.co/xnaqyus1wi',
 'mom',
 'are',
 'you',
 'getting',
 'involved',
 'college',
 'nyou',
 'https://t.co/ek2hwey6pm',
 '@iupuiregatta',
 'four',
 'days',
 'till',
 'regatta',
 'here',
 'what',
 'have',
 'going',
 'today',
 '#yougottaregatta',
 'https://t.co/hf5a3mvypy',
 '@iupuiregatta',
 'four',
 'days',
 'till',
 'regatta',
 'here',
 'what',
 'have',
 'going',
 'today',
 '#yougottaregatta',
 'https://t.co/hf5a3mvypy',
 'regatta',
 'week',
 'finally',
 'here',
 'get',
 'the',
 'spirit',
 'with',
 'these',
 'events',
 'battleships',
 'water',
 'games',
 "s'mores",
 'https://t.co/nw48qhrmpc',
 '@iupuijaguars',
 'pretty',
 'good',
 'weekend',
 'all',
 'around',
 'for',
 '@iupuiwsoc',
 '@iupuimenssoccer',
 'and',
 '@iupuivball',
 'this',
 'past',
 'weeken

In [144]:
## Get most frequent words
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

def get_most_frequent_words(text_series, stop_words = None, ngram_range = (1,2)):
    
    count_vectorizer = CountVectorizer(analyzer = "word",
                                       tokenizer = tweet_tokenizer,
                                       stop_words = stop_words,
                                       ngram_range = ngram_range 
                                      )
    
    term_freq_matrix = count_vectorizer.fit_transform(text_series)
        
    terms = count_vectorizer.get_feature_names()
    term_frequencies = term_freq_matrix.sum(axis = 0).tolist()[0]
    
    
    term_freq_df = (pd.DataFrame(list(zip(terms, term_frequencies)), columns = ["token","count"])
                    .set_index("token")
                    .sort_values("count",ascending = False))
    
    ### Avoiding hashtags and mentions
    data = term_freq_df.index.str.startswith(("@","#"))

    data = [not(data[i]) for i in range(len(data))]
    term_freq_df = term_freq_df[data]
    return term_freq_df

term_freq_df = get_most_frequent_words(df["text"],stop_words = stop_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mchivuku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [145]:
term_freq_df = term_freq_df[:100]

## Most Frequent Hashtags and Screennames

In [146]:
df.columns

Index(['created', 'text', 'sentiment', 'tweet_id', 'truncated', 'user',
       'user_followers_count', 'user_friends_count', 'user_favourites_count',
       'user_statuses_count', 'retweeted_status', 'retweet_count',
       'favorite_count', 'reply_count', 'at_mentions', 'hashtags', 'urls',
       'source', 'lat', 'long', 'tweet_type', 'words'],
      dtype='object')

In [147]:
hashtags = []
screen_names = []
for index, row in df.iterrows():
    if isinstance(row["hashtags"], str):
        hashtags.extend([ t for t in str(row["hashtags"]).split(", ")])
    if isinstance(row["at_mentions"], str):
        screen_names.extend([t for t in str(row["at_mentions"]).split(", ")]) 
    


In [148]:
print("Hashtags: ", len(hashtags),"Screen_names:", len(screen_names))


Hashtags:  557 Screen_names: 2570


#### Build Hashtag and Mentions table

In [149]:
from prettytable import PrettyTable
from collections import Counter
for label, data in (('Screen Name', screen_names),
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count'])
    
    
    ### Merge duplicates
    c = {}
   
    for k in list(data):
        keys = [str.lower(p) for p in c.keys()]
        if str.lower(k) in keys:
            current_keys = list(c.keys())
            match_index = keys.index(str.lower(k))
            match = current_keys[match_index]
            c[match] += 1
        else:
            c[k] = 1
    
    if label == 'Screen Name':
        screenname_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        screenname_df = screenname_df.rename(columns={'index':'screen_name', 0:'count'}).sort_values("count",ascending = False)
        print(screenname_df.head())
    else:
        hashtag_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        hashtag_df = hashtag_df.rename(columns= {'index':"hashtag",0:'count'}).sort_values("count",ascending = False)
        print(hashtag_df.head())
    

     screen_name  count
9   IUPUI         362  
12  paydar        231  
3   IUPUIJaguars  67   
98  IndianaUniv   61   
16  IUPD_IUPUI    49   
            hashtag  count
33  MyIUPUI          33   
9   JagsROAR         32   
4   IUPUI            26   
0   YouGottaRegatta  23   
55  iupuigrad19      21   


In [150]:
hashtag_df.head()

,hashtag,count
33,MyIUPUI,33
9,JagsROAR,32
4,IUPUI,26
0,YouGottaRegatta,23
55,iupuigrad19,21


In [151]:
print(f"Screennames: {screenname_df.shape}, hashtags: {hashtag_df.shape}")

Screennames: (689, 2), hashtags: (219, 2)


### Top 100 - mentions and hashtags

In [152]:
screenname_df, hashtag_df  = screenname_df[:100], hashtag_df[:100]

### Add Created Date


In [153]:
df['created date'] = df["created"].dt.date


## Build Excel Sheets

In [154]:
sentiment_df = df.loc[:,['sentiment']]


In [155]:
#df = df.drop(['Created Date'],axis=1)
df.head()

,created,text,sentiment,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,user_statuses_count,...,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type,words,created date
0,2019-09-17 18:46:00,YOU SUNK MY….\n\n🚢 @iupuidm Battleships\n🚢 8 p.m.\n🚢 Natatorium https://t.co/n7Q9Gj6lSG,Neutral,1174031766677598209,False,IUPUI,36452,3143,5413,27327,...,0,IUPUIdm,NaN,NaN,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,tweet,sunk … battleship .. natatorium,2019-09-17
1,2019-09-17 16:35:24,Tune in tomorrow!👇👇👇 https://t.co/d3UWRrhxTr,Neutral,1173998901914816512,False,IUPUI,36452,3143,5413,27327,...,0,NaN,NaN,https://t.co/d3UWRrhxTr,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,quote,tune tomorrow,2019-09-17
2,2019-09-17 16:06:46,Happy 125th Birthday to our most \nlaw-some school: @IUMcKinney! 🥳 https://t.co/xnaQyus1WI,Neutral,1173991695945367552,False,IUPUI,36452,3143,5413,27327,...,0,IUMcKinney,NaN,https://t.co/xnaQyus1WI,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,quote,happi 125th birthday law-som school,2019-09-17
3,2019-09-17 15:11:33,Mom: Are you getting involved in college?\n\nYou: https://t.co/Ek2hweY6Pm,Neutral,1173977797896880129,False,IUPUI,36452,3143,5413,27327,...,0,NaN,NaN,https://t.co/Ek2hweY6Pm,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,quote,mom get involv colleg,2019-09-17
4,2019-09-17 12:05:17,RT @IUPUIRegatta: FOUR days till Regatta! Here’s what we have going on today. #YouGottaRegatta https://t.co/HF5a3MVypY,Neutral,1173930924628697088,False,IUPUI,36452,3143,5413,27327,...,0,IUPUIRegatta,YouGottaRegatta,NaN,"<a href=""https://about.twitter.com/products/tweetdeck"" rel=""nofollow"">TweetDeck</a>",NaN,NaN,retweet,four day till regatta ’ go today,2019-09-17


<ul>
<li> Twitter Data </li>
<li> User Profile Data </li>
<li> Most Frequent Words </li>
<li> Most Frequent Hashtags </li>
<li> Most Frequent Mentions </li>

</ul>

In [156]:
user_data_df

,user_id,name,screen_name,followers_count,friends_count,favourites_count,statuses_count,total_reach,avg_follower_cnt
0,10086382,IUPUI,IUPUI,35948,3151,5144,27089,10395659,289.19


In [157]:

with pd.ExcelWriter(os.path.join(processed_data_dir,file),engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name=twitter_data_sheetname,index=False)
    user_data_df.to_excel(writer, sheet_name=twitter_user_profile_data_sheetname,index=False)
    term_freq_df.to_excel(writer, sheet_name=most_frequent_words_sheetname)
    screenname_df.to_excel(writer, sheet_name=most_frequent_screennames_sheetname,index=False)
    hashtag_df.to_excel(writer, sheet_name=most_frequent_hashtags_sheetname,index=False)
    writer.save()
    